# Tests
## Import libraries

In [1]:
import os
import sys
import openai
import pandas as pd
from dotenv import load_dotenv


src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

%load_ext autoreload
%autoreload 2

from jobs_tools import tests_helpers, chat_gpt


load_dotenv()
client_async = openai.AsyncOpenAI()

In [9]:
df = pd.read_csv("../data/csv/df_translated.csv")
df.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language,Job Description English
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer,en,We are a dynamic FinTech company headquartered...
1,Austria,Europe,True,True,default,google.com,Mobile Application Developer,Pearson Carter,Austria,"Trabajo.org - Stellenangebote, Arbeit",Lead Mobile Developer | Hyper Growth Startup |...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJNb2JpbGUgQXBwbGljYXRpb24gRG...,2025-01-13 12:08:51 UTC,Android developer,en,Lead Mobile Developer | Hyper Growth Startup |...
2,Austria,Europe,True,True,default,google.com,"Android Developer – Kotlin (Austria based, Hyb...",Bitcoin Devs Company,"Vienna, Austria",Jobs3,Overview:\r\nThe Android Developer – Kotlin po...,NaN,NaN,Contractor,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciDigJ...,2025-01-13 12:08:48 UTC,Android developer,en,Overview:\r\nThe Android Developer – Kotlin po...
3,Austria,Europe,True,True,default,google.com,Android & iOS Developer,ventopay gmbh,Austria,StudySmarter - Talents,Was sind deine Aufgaben?\r\n• Du gestaltest at...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIFx1MDAyNiBpT1MgRG...,2025-01-13 12:20:43 UTC,iOS developer,de,**What are your tasks?** \n- You design attra...
4,Austria,Europe,True,True,default,google.com,iOS Developer Up3 (f/m/d),Drei Österreich,"Vienna, Austria","MyAbility.jobs, Drei., Jobted.at",Do you want to push the frontier of digital se...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIFVwMyAoZi...,2025-01-13 12:20:39 UTC,iOS developer,en,Do you want to push the frontier of digital se...


## Split job descriptions into structured sections
### Columns into text

In [10]:
sample = df.sample(n=30, random_state=42)
#tests_helpers.column_to_txt(column='Job Description English', output="../data/ground truth/job_descriptions_sample.txt", df=sample)

### Load ground truth into DataFrame

In [11]:
test = tests_helpers.txt_to_column(txt_path="../data/ground truth/job_descriptions_ground_truth.txt", df=sample)
test.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language,Job Description English,Ground Truth
1575,Poland,Europe,True,True,default,google.com,Android Developer,Connectis_,"Kórnik, Poland",HitPraca.pl,...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciIsIm...,2025-01-13 12:11:12 UTC,Android developer,pl,**Android Developer** \n**Location:** Poznań ...,1. Platform: Android\n2. Salary: Not mentioned...
324,Canada,Northern America,False,False,default,google.com,iOS Developer (Remote),McAfee,"Waterloo, ON, Canada",Blind,...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIChSZW1vdG...,2025-01-13 12:21:02 UTC,iOS developer,en,Role OverviewMcAfee is searching for an interm...,1. Platform: iOS\n2. Salary: Not mentioned\n3....
1491,Netherlands,Europe,True,True,default,google.com,IOS developer,StarApple,"Eindhoven, Netherlands",Werkzoeken.nl,...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJJT1MgZGV2ZWxvcGVyIiwiY29tcG...,2025-01-13 12:22:59 UTC,iOS developer,nl,**Organization & Department**\n\nAre you inter...,"1. Platform: iOS\n2. Salary: €3,200 to €5,000\..."
1815,Romania,Europe,True,True,default,google.com,Android Developer / CTS Expert,Luxoft,Romania,"Indeed, Jooble, LinkedIn",...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciAvIE...,2025-01-13 12:11:28 UTC,Android developer,en,"Remote Romania, Romania\r\n\r\nAndroid\r\n\r\n...",1. Platform: Android\n2. Salary: Not mentioned...
2174,United Kingdom,Europe,False,False,default,google.com,Embedded Software Developer (Android),Integrity,"Chesterton, United Kingdom",SitePoint,...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJFbWJlZGRlZCBTb2Z0d2FyZSBEZX...,2025-01-13 12:12:03 UTC,Android developer,en,If you have experience developing embedded rea...,1. Platform: Android\n2. Salary: £40-50K plus ...


#### Split ground truth into sections

In [12]:
test["Platform_GT"] = test["Ground Truth"].apply(lambda x: tests_helpers.extract_section(x, r"1\. Platform"))
test["Salary_GT"] = test["Ground Truth"].apply(lambda x: tests_helpers.extract_section(x, r"2\. Salary"))
test["Requirements_GT"] = test["Ground Truth"].apply(lambda x: tests_helpers.extract_section(x, r"3\. Requirements"))
test["Nice to have_GT"] = test["Ground Truth"].apply(lambda x: tests_helpers.extract_section(x, r"4\. Nice to have"))
test["Responsibilities_GT"] = test["Ground Truth"].apply(lambda x: tests_helpers.extract_section(x, r"5\. Responsibilities"))
test["Benefits_GT"] = test["Ground Truth"].apply(lambda x: tests_helpers.extract_section(x, r"6\. Benefits"))

test.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Search Query,Language,Job Description English,Ground Truth,Platform_GT,Salary_GT,Requirements_GT,Nice to have_GT,Responsibilities_GT,Benefits_GT
1575,Poland,Europe,True,True,default,google.com,Android Developer,Connectis_,"Kórnik, Poland",HitPraca.pl,...,Android developer,pl,**Android Developer** \n**Location:** Poznań ...,1. Platform: Android\n2. Salary: Not mentioned...,Android,Not mentioned,- Minimum 3 years of commercial experience as ...,- Familiarity with technologies and tools such...,- Designing and implementing new screens in th...,- Opportunity to participate in integration ev...
324,Canada,Northern America,False,False,default,google.com,iOS Developer (Remote),McAfee,"Waterloo, ON, Canada",Blind,...,iOS developer,en,Role OverviewMcAfee is searching for an interm...,1. Platform: iOS\n2. Salary: Not mentioned\n3....,iOS,Not mentioned,- You can develop iOS applications at an inter...,- Diving deep into lower-level libraries like ...,"(list out or say ""Not mentioned"")\n - Your wo...",Not mentioned
1491,Netherlands,Europe,True,True,default,google.com,IOS developer,StarApple,"Eindhoven, Netherlands",Werkzoeken.nl,...,iOS developer,nl,**Organization & Department**\n\nAre you inter...,"1. Platform: iOS\n2. Salary: €3,200 to €5,000\...",iOS,"€3,200 to €5,000",- 3 years of work experience with Swift\n - B...,- Experience in an environment where cybersecu...,- Primarily focus on backend development for t...,- 28 vacation days + the option to purchase 10...
1815,Romania,Europe,True,True,default,google.com,Android Developer / CTS Expert,Luxoft,Romania,"Indeed, Jooble, LinkedIn",...,Android developer,en,"Remote Romania, Romania\r\n\r\nAndroid\r\n\r\n...",1. Platform: Android\n2. Salary: Not mentioned...,Android,Not mentioned,- Deep expertise in Google CTS tests\n - Stro...,- Docker\n - AUTOSAR (AUTomotive Open System ...,- Work inside an intercultural team of many la...,Not mentioned
2174,United Kingdom,Europe,False,False,default,google.com,Embedded Software Developer (Android),Integrity,"Chesterton, United Kingdom",SitePoint,...,Android developer,en,If you have experience developing embedded rea...,1. Platform: Android\n2. Salary: £40-50K plus ...,Android,£40-50K plus benefits,- Degree in relevant subject\n - Embedded rea...,- Experience of Linux kernel and system progra...,Not mentioned,- An early finish on Fridays\n - Bonus Privat...


### Chat GPT's version

In [13]:
system_prompt = """
You are an AI assistant. Your role is to extract specific information from job descriptions and format them in a strict structure.
"""

user_prompt = """
I will provide a job description. Please extract and present the information in **this exact order**:

1. Platform: (Android/iOS/Cross-platform)
2. Salary: (If stated; otherwise "Not mentioned")
3. Requirements: (verbatim from the job description or "Not mentioned")
4. Nice to have: (verbatim or "Not mentioned")
5. Responsibilities: (verbatim or "Not mentioned")
6. Benefits: (verbatim or "Not mentioned")

**Guidelines**:
- **DO NOT reword, paraphrase, or summarize** any part of the job description. Copy the sentences exactly as they appear.
- Combine all mandatory or required skill sections (e.g., "Requirements," "Skills," "Key Technologies," "About You") under **Requirements**.
- If the job description specifically says something is "a plus," "beneficial," or otherwise indicates it’s optional, place it under "Nice to have" even if it appears under a "Requirements" heading in the job description.
- If there is an "About you" or "About Role" section (or similar) that describs duties or tasks, include those under "Responsibilities".
- If the information is not in the job description, write "Not mentioned" for that section.
- For multiple platforms (e.g., Android, iOS), list them all in **Platform** and use headings under Requirements (and other sections, if needed) like "General Requirements:", "For Android Developers:", "For iOS Developers:".
- Present your answer **only** in the format above.

---
Here is the job description:
"""

df_extracted_test = await chat_gpt.chatgpt_async(
    input_column_name="Job Description English", 
    output_column_name="Job Description Extracted",
    input_text_length=None,
    output_text_length=None,
    num_rows=None,  
    df=test.copy(), 
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    gpt_model="gpt-4o-2024-11-20",
    client=client_async,
    batch_size=5,
    concurrency_limit=10,
    cache_file="../data/cache/job_description_extracted_cache_gpt-4o-2024-11-20.json"
)

df_extracted_test.head()

Processing Batches: 100%|██████████| 7/7 [00:01<00:00,  6.11it/s]


,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Language,Job Description English,Ground Truth,Platform_GT,Salary_GT,Requirements_GT,Nice to have_GT,Responsibilities_GT,Benefits_GT,Job Description Extracted
1575,Poland,Europe,True,True,default,google.com,Android Developer,Connectis_,"Kórnik, Poland",HitPraca.pl,...,pl,**Android Developer** \n**Location:** Poznań ...,1. Platform: Android\n2. Salary: Not mentioned...,Android,Not mentioned,- Minimum 3 years of commercial experience as ...,- Familiarity with technologies and tools such...,- Designing and implementing new screens in th...,- Opportunity to participate in integration ev...,1. Platform: Android \n2. Salary: Not mention...
324,Canada,Northern America,False,False,default,google.com,iOS Developer (Remote),McAfee,"Waterloo, ON, Canada",Blind,...,en,Role OverviewMcAfee is searching for an interm...,1. Platform: iOS\n2. Salary: Not mentioned\n3....,iOS,Not mentioned,- You can develop iOS applications at an inter...,- Diving deep into lower-level libraries like ...,"(list out or say ""Not mentioned"")\n - Your wo...",Not mentioned,1. Platform: iOS/macOS \n2. Salary: Not menti...
1491,Netherlands,Europe,True,True,default,google.com,IOS developer,StarApple,"Eindhoven, Netherlands",Werkzoeken.nl,...,nl,**Organization & Department**\n\nAre you inter...,"1. Platform: iOS\n2. Salary: €3,200 to €5,000\...",iOS,"€3,200 to €5,000",- 3 years of work experience with Swift\n - B...,- Experience in an environment where cybersecu...,- Primarily focus on backend development for t...,- 28 vacation days + the option to purchase 10...,"1. Platform: iOS \n2. Salary: €3,200 to €5,00..."
1815,Romania,Europe,True,True,default,google.com,Android Developer / CTS Expert,Luxoft,Romania,"Indeed, Jooble, LinkedIn",...,en,"Remote Romania, Romania\r\n\r\nAndroid\r\n\r\n...",1. Platform: Android\n2. Salary: Not mentioned...,Android,Not mentioned,- Deep expertise in Google CTS tests\n - Stro...,- Docker\n - AUTOSAR (AUTomotive Open System ...,- Work inside an intercultural team of many la...,Not mentioned,1. Platform: Android \n2. Salary: Not mention...
2174,United Kingdom,Europe,False,False,default,google.com,Embedded Software Developer (Android),Integrity,"Chesterton, United Kingdom",SitePoint,...,en,If you have experience developing embedded rea...,1. Platform: Android\n2. Salary: £40-50K plus ...,Android,£40-50K plus benefits,- Degree in relevant subject\n - Embedded rea...,- Experience of Linux kernel and system progra...,Not mentioned,- An early finish on Fridays\n - Bonus Privat...,1. Platform: Android \n2. Salary: £40-50K plu...


#### Split Chat GPT's version into sections

In [14]:
df_extracted_test["Platform_GPT"] = df_extracted_test["Job Description Extracted"].apply(lambda x: tests_helpers.extract_section(x, r"1\. Platform"))
df_extracted_test["Salary_GPT"] = df_extracted_test["Job Description Extracted"].apply(lambda x: tests_helpers.extract_section(x, r"2\. Salary"))
df_extracted_test["Requirements_GPT"] = df_extracted_test["Job Description Extracted"].apply(lambda x: tests_helpers.extract_section(x, r"3\. Requirements"))
df_extracted_test["Nice to have_GPT"] = df_extracted_test["Job Description Extracted"].apply(lambda x: tests_helpers.extract_section(x, r"4\. Nice to have"))
df_extracted_test["Responsibilities_GPT"] = df_extracted_test["Job Description Extracted"].apply(lambda x: tests_helpers.extract_section(x, r"5\. Responsibilities"))
df_extracted_test["Benefits_GPT"] = df_extracted_test["Job Description Extracted"].apply(lambda x: tests_helpers.extract_section(x, r"6\. Benefits"))

df_extracted_test.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Nice to have_GT,Responsibilities_GT,Benefits_GT,Job Description Extracted,Platform_GPT,Salary_GPT,Requirements_GPT,Nice to have_GPT,Responsibilities_GPT,Benefits_GPT
1575,Poland,Europe,True,True,default,google.com,Android Developer,Connectis_,"Kórnik, Poland",HitPraca.pl,...,- Familiarity with technologies and tools such...,- Designing and implementing new screens in th...,- Opportunity to participate in integration ev...,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,- Minimum 3 years of commercial experience as ...,- Familiarity with technologies and tools such...,- Designing and implementing new screens in th...,- Opportunity to participate in integration ev...
324,Canada,Northern America,False,False,default,google.com,iOS Developer (Remote),McAfee,"Waterloo, ON, Canada",Blind,...,- Diving deep into lower-level libraries like ...,"(list out or say ""Not mentioned"")\n - Your wo...",Not mentioned,1. Platform: iOS/macOS \n2. Salary: Not menti...,iOS/macOS,Not mentioned,- You can develop iOS applications at an inter...,- Diving deep into lower-level libraries like ...,- Dive deep into anti-censorship technologies ...,Not mentioned
1491,Netherlands,Europe,True,True,default,google.com,IOS developer,StarApple,"Eindhoven, Netherlands",Werkzoeken.nl,...,- Experience in an environment where cybersecu...,- Primarily focus on backend development for t...,- 28 vacation days + the option to purchase 10...,"1. Platform: iOS \n2. Salary: €3,200 to €5,00...",iOS,"€3,200 to €5,000",- 3 years of work experience with Swift \n ...,- Experience in an environment where cybersecu...,- You’ll primarily focus on backend developmen...,- 28 vacation days + the option to purchase 10...
1815,Romania,Europe,True,True,default,google.com,Android Developer / CTS Expert,Luxoft,Romania,"Indeed, Jooble, LinkedIn",...,- Docker\n - AUTOSAR (AUTomotive Open System ...,- Work inside an intercultural team of many la...,Not mentioned,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,- Deep expertise in Google CTS tests \n - S...,- Docker \n - AUTOSAR (AUTomotive Open Syst...,- Work inside an intercultural team of many la...,Not mentioned
2174,United Kingdom,Europe,False,False,default,google.com,Embedded Software Developer (Android),Integrity,"Chesterton, United Kingdom",SitePoint,...,- Experience of Linux kernel and system progra...,Not mentioned,- An early finish on Fridays\n - Bonus Privat...,1. Platform: Android \n2. Salary: £40-50K plu...,Android,£40-50K plus benefits,- Degree in relevant subject \n - Embedded ...,- Experience of Linux kernel and system progra...,- Develop and support the software running on ...,- An early finish on Fridays \n - Bonus \n...


### Calculate Fuzzy Scores

1. Token Sort Ratio (Handling Word Order Differences)<br>
If two sentences contain the same words but in different order, Levenshtein distance alone might give a low similarity score. Instead, we:
- Split both sentences into words.
- Sort them alphabetically.
- Recalculate Levenshtein Distance.

2. Token Set Ratio (Handling Partial Overlaps)<br>
If one string is a subset of another, Token Set Ratio helps:
- Convert both sentences into sets of unique words.
- Compare only the common words.

In [15]:
tests_helpers.compare_fuzzy_sections(df_extracted_test)

Average Token Set Ratio per section:
Platform: 98.33
Salary: 97.35
Requirements: 98.61
Nice to have: 93.38
Responsibilities: 90.88
Benefits: 91.85

Overall Average Token Set Ratio: 95.07
